In [ ]:
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
from ukbb_recessive.regression.regressions import read_results_excel
import ukbb_recessive.regression.plotting as plotting

sns.set_style("whitegrid")

# Add fonts
plotting.add_fonts(['../../../data/fonts'])
plotting.configure_matplotlib()

# Prepare datasets

## Phenotypes regressions on recessive vs other genes

In [ ]:
tables_folder = '../../../data/tables'
output_folder = '../../../data/plots'

use_dataset = 'Roulette'

results_path = f"{tables_folder}/table_phenotype_analysis.xlsx"

# load flat table
phenotypes = read_results_excel(results_path, flatten_multiindex=True)

# leave only necessary s_het effect
phenotypes = phenotypes[phenotypes['feature'].str.contains("s_het")]

# add odds ratio distance
phenotypes = plotting.add_odds_ratio_intervals(phenotypes)
phenotypes = plotting.add_effect_size_intervals(phenotypes)

# filter out dataset
if use_dataset:
    phenotypes = phenotypes[phenotypes['dataset'] == use_dataset]

In [ ]:
phenotypes['analysis'].unique().tolist()

In [ ]:
# Dictionary to rename target columns
renaming_dict_target = {
    'is_blond': 'Hair color', 
    'years_of_edu': 'Years of\neducation',
    'diagnosis_total_ICD10_cnt_log': 'Diagnoses\ncount (log)',
    'ICD_infertility': 'Infertility',
    'living_with_a_partner': 'Has partner',
    'fluid_intelligence_score': 'Fluid\nintelligence',
    'childlessness': 'Childlessness'
}

# select targets of interest
phenotypes = phenotypes[phenotypes['target'].isin(renaming_dict_target.keys())].copy()
phenotypes['target'] = phenotypes['target'].apply(lambda x: renaming_dict_target.get(x, x))

# select gender of interest
phenotypes = phenotypes[phenotypes['gender'] == 'all']

# select analyses
renaming_dict_analyses = {
    'phenotype regressions on [s_het_recessive_all]' : 'PLPs in recessive genes',
    'phenotype regressions on [s_het_lof_without_AR]' : 'Singleton LoFs in non-recessive genes'
}

phenotypes = phenotypes[phenotypes['analysis'].isin(renaming_dict_analyses.keys())].copy()
phenotypes['analysis'] = phenotypes['analysis'].apply(lambda x: renaming_dict_analyses.get(x, x))

phenotypes.head(3)

## Phenotypre regressions on synonymous variants

In [ ]:
results_path = f"../../../data/tables/sampling/synonymous/table_synonymous_on_phenotype_analysis.xlsx"

# load flat table
synonymous = read_results_excel(results_path, flatten_multiindex=True)

# leave only necessary s_het effect
synonymous = synonymous[synonymous['feature'].str.contains("s_het")]

# add odds ratio distance
synonymous = plotting.add_odds_ratio_intervals(synonymous)
synonymous = plotting.add_effect_size_intervals(synonymous)

# filter out dataset
if use_dataset:
    synonymous = synonymous[synonymous['dataset'] == use_dataset]

In [ ]:
# select targets of interest
synonymous = synonymous[synonymous['target'].isin(renaming_dict_target.keys())].copy()
synonymous['target'] = synonymous['target'].apply(lambda x: renaming_dict_target.get(x, x))

# select gender of interest
synonymous = synonymous[synonymous['gender'] == 'all']

synonymous.head(3)

# Plot

In [ ]:
# Create a function to format numbers with a comma as a thousand separator
def format_thousands(x):
    return f'{x:,.0f}'  # Use , as a thousand separator and format as integers

def plot_phenotypes(ax, y_order, plot_entity='odds_ratio', legend_loc='upper left'):

    group_order = ['Singleton LoFs in non-recessive genes', 'PLPs in recessive genes'][::-1]

    legend_kwargs = {
        "bbox_to_anchor": (0.0, 1., 1, .102), 
        "frameon": False,
        "mode": "expand", 
        "ncol": 1, 
        "labelspacing": 0.1, 
        "markerfirst": False, 
        'fontsize': plotting.MEDIUM_SIZE
    }

    colors=['#B048B5', '#046307',]

    vline_loc = 1 if plot_entity == 'odds_ratio' else 0

    plotted_data = plotting.plot_errorbar_grouped(df=phenotypes[phenotypes['target'].isin(y_order)].copy(), 
                                        axis=ax, y_column='target', group_column = 'analysis',  
                                        legend_loc=legend_loc, 
                                        legend_kwargs=legend_kwargs,
                                        group_scale=0.03, 
                                        y_scale=0.1,
                                        colors=colors,
                                        y_order=y_order, 
                                        group_order=group_order, 
                                        plot_entity=plot_entity, 
                                        vertical_loc=vline_loc, 
                                        horizontal=False)


    # plotting.configure_axis(ax, x_label='Odds ratio (99% CI)', y_label=None, ymargin=0.12)

    # Display p-values or any other information in the sidebar
    p_values = plotted_data['p_value_corrected_pretty']
    or_values = plotted_data[plot_entity] 
    positions = plotted_data['y']

    plot_loc = plotted_data[f'{plot_entity}'] - plotted_data[f'{plot_entity}_lower_distance']

    for i, (loc, p, oddsr, pos) in enumerate(zip(plot_loc, p_values, or_values, positions)):
        ax.text(loc, pos+0.005, f'{round(oddsr, 2)}   {p}', va='center', size=plotting.MEDIUM_SIZE)

def plot_synonymous(ax, targets, plot_entity='odds_ratio', vline_loc=1):

    # select necessary targets
    data_syns = synonymous[synonymous['target'].isin(targets)].copy()
    data_plps = phenotypes[phenotypes['target'].isin(targets) & (phenotypes['analysis'] == 'PLPs in recessive genes')].copy()

    # sort dataset according to the target order
    data_syns.target = pd.Categorical(data_syns.target, categories=targets)
    data_syns = data_syns.sort_values('target')

    data_plps.target = pd.Categorical(data_plps.target, categories=targets)
    data_plps = data_plps.sort_values('target')

    # plot neutral line
    ax.vlines(x=vline_loc, ymin=-3, ymax=10, color='grey', linestyle='--', alpha=0.5, linewidth=0.5)
    
    vp = sns.violinplot(data=data_syns[(data_syns['gender'] == 'all')], y='target', x=plot_entity, hue='target',
                ax=ax, linewidth=0.5, linecolor='k', palette=['#a0b4ff'])

    # plot PLPs estimation
    pp = sns.pointplot(data=data_plps[(data_plps['gender'] == 'all') & (data_plps['target'].isin(targets))], y='target', x=plot_entity,
                    linestyle='none', markers='o', ax=ax, color='#ffa260', markersize=3, markeredgewidth=0.5, markeredgecolor='k')

In [ ]:
import matplotlib.ticker as ticker

cm = 1/2.54  # centimeters in inches
k = 1.
fig = plt.figure(constrained_layout=True, figsize=(8.8*cm*k, 6*cm*k))

# divide subfigure into left and right parts
subfigs = fig.subfigures(nrows=1, ncols=2, width_ratios=[1, 1], wspace=0.03)

# plot phenotypes results
ax_left = subfigs[0].subplots(3, 1)

# years of education
plot_phenotypes(ax_left[0], y_order=['Years of\neducation'], plot_entity='effect', legend_loc=None)
plotting.configure_axis(ax_left[0], x_label='Effect size',  ymargin=1, format_x=True)
ax_left[0].set_xticks([-3, -2, -1, 0])
ax_left[0].set_xticklabels([-3, -2, -1, 0])

# fluid intelligence
plot_phenotypes(ax_left[1], y_order=['Fluid\nintelligence'], plot_entity='effect',legend_loc=None)
plotting.configure_axis(ax_left[1], xlim=(-2, 0.1), x_label='Effect size',  ymargin=1, format_x=True)
ax_left[1].set_xticks([-2.0, -1.5, -1.0, -0.5, 0.0])
ax_left[1].set_xticklabels([-2.0, -1.5, -1.0, -0.5, 0.0])

# diagnoses count
plot_phenotypes(ax_left[2], y_order=['Diagnoses\ncount (log)'], plot_entity='effect', legend_loc=None)
plotting.configure_axis(ax_left[2], xlim=(-0.01, 0.6), x_label='Effect size',  ymargin=1, format_x=True)
ax_left[2].set_xticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
ax_left[2].set_xticklabels([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

# # infertility
# plot_phenotypes(ax_left[3], y_order=['Infertility'], plot_entity='odds_ratio', legend_loc=None)
# plotting.configure_axis(ax_left[3], xlim=(-0.1, 6), x_label=None,  ymargin=0.6, format_x=True)
# ax_left[3].set_xticks([0, 1, 2, 3, 4, 5, 6])
# ax_left[3].set_xticklabels([0, 1, 2, 3, 4, 5, 6])

# plot synonymous
ax_right = subfigs[1].subplots(1, 1)

plot_synonymous(ax_right, targets = ['Childlessness', 'Years of\neducation', 'Fluid\nintelligence', 
                                     'Diagnoses\ncount (log)', 'Infertility', 'Hair color'], plot_entity='effect', vline_loc=0)

plotting.configure_axis(ax_right, x_label='Effect size', y_label='Phenotype', xlim=None, ymargin=0.02, format_x=True)

# save figure
plt.savefig(f"{output_folder}/figure_2.pdf", format="pdf", bbox_inches="tight")